Agent Example

In [182]:
from pymongo import MongoClient
import os
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain.prompts import PromptTemplate


Remember to create vector index in mongodb and when uploading articles using newsMongo.py, make sure vector embedding is called "embedding":

vector index :{
  "fields": [
    {
      "numDimensions": 1536,
      "path": "embedding",
      "similarity": "cosine",
      "type": "vector"
    }
  ]
}


Scenario 1 for when user asking any news related question, it will vector db search and return the content for llm to digest and answer

In [183]:
DB_NAME = "news_articles"
COLLECTION_NAME = "newsArticleCollection"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

prompt_template = """ 
    Answer user's questions with the information you know.

    {context}

    Question: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo-0125',
    temperature=0.0
)

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
    os.getenv("MONGODB_URI"),
    DB_NAME + "." + COLLECTION_NAME,
    OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536),
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key = "content"
)

qa_retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= qa_retriever,
    # return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    verbose=True
)

Scenario 2 for when user ask agent to recommend some news, itll return the title and link result for now. max article 5 since vector_search returns top 5

In [184]:

# Initialize MongoDB
mongo_client = MongoClient(os.getenv("MONGODB_URI"))
db = mongo_client.get_database("news_articles")
collection = db.get_collection("newsArticleCollection")

query_embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536) # model used to embed user queries

def vector_search(query, collection):
    query_embedding = query_embeddings.embed_query(query)
    pipeline = [
        {
            "$vectorSearch": { # $vectorSearch is the specific function name
                "index": "vector_index", # The search index I created on MongoDB
                "queryVector": query_embedding, # The embedded query from the user that is used for searching
                "path": "embedding", # The relevant field of the document that is used for searching (in this case the full text of the news article)
                "limit": 5, # How many results you want the vectorSearch to show
                "numCandidates": 100 # How many documents you want vectorSearch to consider when searching
            }
        }
    ]
    
    results = collection.aggregate(pipeline) # executing the search
    return list(results) # compile results into a list

def recommend(query):
    get_knowledge = vector_search(query, collection) # Get the output from MongoDB after vector searching

    search_result = ''
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Link: {result.get('url', 'N/A')}, \n"
    return search_result

Scenario 3 Right now if there is date that matches dd Month yyyy or dd/mm/yyy or dd/mm/yy format itll work

In [185]:
import re
from datetime import datetime
def extract_dates(text):
    # Define regex patterns for different date formats
    patterns = [r'(\d{1,2}\s\w+\s\d{4})',  # dd Month yyyy
                r'(\d{2}/\d{2}/\d{4})',    # dd/mm/yyyy
                r'(\d{2}/\d{2}/\d{2})']    # dd/mm/yy
    
    # Define corresponding date parsing formats
    parsing_formats = ["%d %B %Y", "%d/%m/%Y", "%d/%m/%y"]
    
    formatted_dates = []

    # Split the dates input if there is 2 dates, agent will give us something like 13/03/2024, 14/03/2024 so we split by " " to get both dates
    dates = text.split(" ")
    for date_text in dates:
        matched = False
        # Try to match each pattern and parse dates
        for pattern, parsing_format in zip(patterns, parsing_formats):
            pattern_matched = re.findall(pattern, date_text.strip())
            if pattern_matched and not matched: # If a match is found and not already matched
                matched = True
                for match in pattern_matched:
                    date_obj = datetime.strptime(match, parsing_format)
                    formatted_dates.append(date_obj.strftime("%Y-%m-%dT%H:%M:%S"))
    
    #if only one date appears, means user want articles on that day so i need 2 inputs from e.g. 13/03/24 to 13/03/24
    if len(formatted_dates) == 1:
        formatted_dates = [formatted_dates[0],formatted_dates[0]]
    return formatted_dates

import readDb_Functions
def get_date_specific_articles(query):
    
    mongo_client = MongoClient(os.getenv("MONGODB_URI"))
    db = mongo_client.get_database("news_articles")
    collection = db.get_collection("newsArticleCollection")
    print(extract_dates(query))
    return extract_dates(query)
    #i removed the readDb function to save token, if there is date extracted, highly likely its good to go
    # return readDb_Functions.getNews(collection, ['All'], extract_dates(query))

Different Scenarios for QA handling 

In [186]:
from langchain.agents import Tool

tools = [
    #scenario #2
    #This works e.g. "Give me news on date 13/03/24 to 15/03/24"
    Tool(
        name='Date period',
        func=get_date_specific_articles,
        description=(
            
            """
            use this tool when user mentioned a date. 
            only if user give date such as dd Month yyyy or dd/mm/yyy. 
            for example could be 30 May 2000 or 24/12/19.
            """
        )
    ),
    #scenario #1
    #This works e.g. "What is Saudi Arabia doing"
    Tool(
        name='Ask',
        func=qa.run,
        description=(
            """
            Use this tool when user ask questions
            do not use this tool if user mentioned date or words like "recommend" or "suggest", do not summarize observation.
            """
        )
    ),
    
    #scenario #3
    #This works e.g. "Recommend me 3 Apple news"
    Tool(
        name='Recommend',
        func=recommend,
        description=(
            """
            The input or user query must contain the word "recommend". If "recommend" does not
            exist in the query or input, then use "Ask" Tool
            """
        )
    )
]

In [187]:
from langchain.agents import initialize_agent

# template = """You are a bot that will either provide news recommendations or answer questions related to the news asked by users.
#     You are speaking to a professional so keep the answer informative and concise. 
    
# {chat_history}
# Human: {human_input}
# Chatbot: Always start output with "Here is my answer":"""
# prompt = PromptTemplate(input_variables=["chat_history", "human_input"],template=template)
# chat completion llm
llm2 = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo-1106',
    temperature=0.0
)

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent = initialize_agent(
    # agent='chat-conversational-react-description',
    agent = 'conversational-react-description',
    tools=tools,
    llm=llm2,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate', 
    memory=conversational_memory,
    # prompt = prompt
)

In [188]:
agent.agent.llm_chain.prompt.template = agent.agent.llm_chain.prompt.template + """
do not use tools if user ask about previous message or elaborate more on previous message or repeat previous message otherwise use "Ask" tool if user ask questions
"""

In [189]:
print(agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [190]:
agent("What is UK government doing")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Ask
Action Input: What is UK government doing

> Entering new RetrievalQA chain...

> Finished chain.

Observation: The UK government is investing heavily in quantum technology and space industries to support startups and drive innovation. They have provided significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, the UK government has launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies.
Thought:AI: The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing sign

{'input': 'What is UK government doing',
 'chat_history': [],
 'output': "The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies."}

In [191]:
agent("Give me news on date 13/03/24 to 15/03/24") #not done



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Date period
Action Input: 13/03/24 to 15/03/24['2024-03-13T00:00:00', '2024-03-15T00:00:00']

Observation: ['2024-03-13T00:00:00', '2024-03-15T00:00:00']
Thought:Thought: Do I need to use a tool? No
AI: I'm sorry, but I don't have access to real-time news updates. Is there anything else I can assist you with?

> Finished chain.


{'input': 'Give me news on date 13/03/24 to 15/03/24',
 'chat_history': [HumanMessage(content='What is UK government doing'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies.")],
 'output': "I'm sorry, but I don't have access to real-time news updates. Is there anything else I can assist you with?"}

In [198]:
agent("Recommend me 3 news related to artificial intelligence")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Recommend
Action Input: 3 news related to artificial intelligence
Observation: Title: This startup taps quantum, AI to fast-track discovery of new materials, Link: https://thenextweb.com/news/this-startup-taps-quantum-ai-to-fast-track-discovery-of-new-materials, 
Title: Workplace AI negatively impacts quality of life, study finds, Link: https://readwrite.com/workplace-ai-negatively-impacts-quality-of-life-study-finds/, 
Title: The UK is improving productivity through new technologies and industrial digital transformation, Link: https://www.businessinsider.com/sc/how-the-uk-is-helping-the-manufacturing-sector-improve-productivity, 
Title: New AI and 5G advancements will usher in the era of edge computing on smartphones, autonomous cars, and more, Link: https://www.businessinsider.com/ai-edge-computing-5g-cloud-artificial-intelligence-2024-3, 
Title: Future-first thinking, Link: https://www.businessinsi

{'input': 'Recommend me 3 news related to artificial intelligence',
 'chat_history': [HumanMessage(content='Can you repeat the second news article'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies. If you have any other questions or need further information, feel free to ask!"),
  HumanMessage(content='Can you tell me more about the first news article'),
  AIMessage(content="The first news article

In [193]:
agent('Can you repeat the second news article')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies. If you have any other questions or need further information, feel free to ask!

> Finished chain.


{'input': 'Can you repeat the second news article',
 'chat_history': [HumanMessage(content='What is UK government doing'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies."),
  HumanMessage(content='Give me news on date 13/03/24 to 15/03/24'),
  AIMessage(content="I'm sorry, but I don't have access to real-time news updates. Is there anything else I can assist you with?"),
  HumanMessage(content='R

In [194]:
agent('Can you tell me more about the first news article')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Ask
Action Input: Can you tell me more about the first news article

> Entering new RetrievalQA chain...

> Finished chain.

Observation: The first news article discusses a German startup called Quantistry that is using a mix of quantum tech, physics-based simulations, and machine learning to fast-track the discovery of sustainable materials for green technologies. The company has recently secured €3 million in funding, including backing from Chemovator, the business incubator of BASF. Quantistry's cloud-based platform allows users to determine material and molecular properties in simulation, speeding up the R&D process and making it more accessible to companies. The funding will be used to further develop the simulation platform and secure new customers.
Thought:Thought: Do I need to use a tool? No
AI: The first news article discusses a German startup called Quantistry that is using a mix of quantum 

{'input': 'Can you tell me more about the first news article',
 'chat_history': [HumanMessage(content='What is UK government doing'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies."),
  HumanMessage(content='Give me news on date 13/03/24 to 15/03/24'),
  AIMessage(content="I'm sorry, but I don't have access to real-time news updates. Is there anything else I can assist you with?"),
  HumanMessage

In [195]:
agent("News about National Cyber Director")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Ask
Action Input: News about National Cyber Director

> Entering new RetrievalQA chain...

> Finished chain.

Observation: The National Cyber Director has urged the largest players in emerging technologies to adopt safer programming languages to reduce memory safety vulnerabilities. This advice was released as part of a new report titled “Back to the Building Blocks: A Path Toward Secure and Measurable Software.” The ONCD advises the President of the United States on cybersecurity policy and strategy. The Biden-Harris Administration has received recognition for reaffirming vulnerabilities that must be considered in lockstep with technological advancements. The ONCD has received support from industry leaders, academia, and the private sector for this initiative.
Thought:Thought: Do I need to use a tool? No
AI: The National Cyber Director has urged the largest players in emerging technologies to adopt s

{'input': 'News about National Cyber Director',
 'chat_history': [HumanMessage(content='What is UK government doing'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies."),
  HumanMessage(content='Give me news on date 13/03/24 to 15/03/24'),
  AIMessage(content="I'm sorry, but I don't have access to real-time news updates. Is there anything else I can assist you with?"),
  HumanMessage(content='Recom

In [196]:
agent('Can you repeat the fourth news article in the previous message')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The fourth news article in the previous message discussed the National Cyber Director urging the largest players in emerging technologies to adopt safer programming languages to reduce memory safety vulnerabilities. This advice was released as part of a new report titled “Back to the Building Blocks: A Path Toward Secure and Measurable Software.” The ONCD advises the President of the United States on cybersecurity policy and strategy. The Biden-Harris Administration has received recognition for reaffirming vulnerabilities that must be considered in lockstep with technological advancements. The ONCD has received support from industry leaders, academia, and the private sector for this initiative.

> Finished chain.


{'input': 'Can you repeat the fourth news article in the previous message',
 'chat_history': [HumanMessage(content='Give me news on date 13/03/24 to 15/03/24'),
  AIMessage(content="I'm sorry, but I don't have access to real-time news updates. Is there anything else I can assist you with?"),
  HumanMessage(content='Recommend me news related to artificial intelligence'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these em

In [197]:
agent('Can you elaborate the fourth news article in the previous message')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The fourth news article in the previous message discussed the National Cyber Director urging the largest players in emerging technologies to adopt safer programming languages to reduce memory safety vulnerabilities. This advice was released as part of a new report titled “Back to the Building Blocks: A Path Toward Secure and Measurable Software.” The ONCD advises the President of the United States on cybersecurity policy and strategy. The Biden-Harris Administration has received recognition for reaffirming vulnerabilities that must be considered in lockstep with technological advancements. The ONCD has received support from industry leaders, academia, and the private sector for this initiative.

> Finished chain.


{'input': 'Can you elaborate the fourth news article in the previous message',
 'chat_history': [HumanMessage(content='Recommend me news related to artificial intelligence'),
  AIMessage(content="The UK government is investing in quantum technology and space industries to support startups and drive innovation. They are providing significant funding for quantum technology startups and space companies, with the aim of boosting growth, creating jobs, and fostering technological advancements. Additionally, they have launched programs like Made Smarter Adoption to support manufacturing SMEs in adopting digital technologies and improving productivity. The government's focus on supporting technology development within the UK is part of a long-term plan to make the country a world leader in these emerging technologies. If you have any other questions or need further information, feel free to ask!"),
  HumanMessage(content='Can you repeat the second news article'),
  AIMessage(content="The UK g

In [154]:
agent('2+2 =?')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The answer to 2+2 is 4. If you have any other questions or need further assistance, feel free to ask!

> Finished chain.


{'input': '2+2 =?',
 'chat_history': [HumanMessage(content='Give me news on date 13/03/24 to 15/03/24'),
  AIMessage(content="I'm sorry, I don't have access to real-time news updates. I recommend checking a reliable news source for the latest news on the dates you mentioned."),
  HumanMessage(content='Recommend me news related to artificial intelligence'),
  AIMessage(content="Here are some news articles related to artificial intelligence that you might find interesting:\n1. Title: Workplace AI negatively impacts quality of life, study finds, Link: [Workplace AI negatively impacts quality of life, study finds](https://readwrite.com/workplace-ai-negatively-impacts-quality-of-life-study-finds/)\n2. Title: This startup taps quantum, AI to fast-track discovery of new materials, Link: [This startup taps quantum, AI to fast-track discovery of new materials](https://thenextweb.com/news/this-startup-taps-quantum-ai-to-fast-track-discovery-of-new-materials)\n3. Title: The UK is improving produc

OpenAI example

In [177]:

# import os
# from openai import OpenAI

# system_message = {"role": "system", 
#                   "content": """You are a bot that will either provide news recommendations, news summaries or answer questions related to the news asked by users.
#     You are speaking to a professional so keep the answer informative and concise.

#     You are given an article(s) to summarize. Please respond with the following using information given.
#     Title: <Title Name>
#     Website Link: <Link of Website>
#     Date of Article: <Get the latest date of publication>
#     Names to note: <Names of Company and people mentioned within the article>
#     Key Topic: <Key topic of this article>
#     Sentiment: <conduct sentiment analysis and let them know the sentiment>
#     Trends & Statistics: <Include any trends and statistics found, make it short and do not repeat it in summary>
#     Summary: <The summarised version of the article>

#         Assistant:"""}

# conversation = []
# conversation.append(system_message)

# client = OpenAI(
#     # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key=os.getenv("OPENAI_API_KEY"),
# )

# def user_query(query):
#     #add user query to conversation for gpt's response 
#     conversation.append({"role": "user", "content": query})
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo-1106",
#         messages= conversation
#         # temperature=0.7,
#     )
#     #add bot reply to conversation
#     conversation.append({"role": "assistant", "content": response.choices[0].message.content})
#     return response.choices[0].message.content.strip()

# from newsMongo import urlScrapeAndStore
# #link test
# url = ['<https://www.cnbc.com/2023/04/30/cloud-providers-amazon-microsoft-and-google-face-spending-cuts.html>']
# output = ''
# articles = []
# for x in url:
#     article = urlScrapeAndStore(x)
#     articles.append(article)
#     output = user_query(str(articles))
#     print("Here is output \n")
#     print(output)
#     print("\n")

# # response = user_query("""Summarise this Coral Bay Nickel Corporation donated uniforms and medical kits to Palawan’s barangay health workers on Monday, an initiative enhancing local healthcare support.

# # The Provincial Information Office said Vice Governor Leoncio Ola and Board Member Marivic Roxas, chairperson of the Committee on Health and Social Services of the Sangguniang Panlalawigan, received the donations.

# # Ernesto Llacuna, community relations manager of CBNC, facilitated the distribution of approximately 2,000 barangya health workers (BHW) uniforms, 150 BP monitors, and 150 thermometers to health workers in the southern region of the province.

# # Attendees included Board Members Juan Antonio Alvarez and Luzviminda Bautista, Dr. Justyn Barbosa, Medical Specialist IV, representing Provincial Health Officer Dr. Faye Erika Q. Labrador, and CVHW Manager Jenevil V. Tombaga.
# #                       """)
# # print(response)

# # print("Here is the conversation with template but with user and chatbot history \\n")
# # print(conversation)

# #can delete/edit role: system basically the template by del conversation[0] or conversation[0]['content'] = new template
# # del conversation[0]
# conversation[0]['content'] = """new template"""

# # print("Here is the conversation without template but with user and chatbot history \n")
# # print(conversation)